# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import nltk
import re
import pickle

# Download necessary NLTK data
nltk.download('punkt')       # Download the 'punkt' tokenizer data
nltk.download('wordnet')     # Download the 'wordnet' corpus for lemmatization
nltk.download('stopwords')   # Download stopwords if you plan to use them

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

# Define feature and target variables X and Y
X = df['message']
Y = df.iloc[:, 4:]  # Assuming the first 4 columns are not target categories

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Lemmatize and remove stop words
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token).strip() for token in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# This pipeline uses TF-IDF and MultiOutputClassifier with RandomForest
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
# Train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Predict on test data
Y_pred = pipeline.predict(X_test)

In [8]:
# Report the f1 score, precision, and recall for each output category
for i, column in enumerate(Y.columns):
    print(f'Category: {column}\n', classification_report(Y_test[column], Y_pred[:, i]))


Category: related
              precision    recall  f1-score   support

          0       0.64      0.39      0.48      1205
          1       0.83      0.93      0.88      4005
          2       0.54      0.38      0.45        34

avg / total       0.79      0.80      0.79      5244

Category: request
              precision    recall  f1-score   support

          0       0.87      0.98      0.93      4317
          1       0.82      0.34      0.48       927

avg / total       0.86      0.87      0.85      5244

Category: offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5226
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5244

Category: aid_related
              precision    recall  f1-score   support

          0       0.73      0.87      0.79      3074
          1       0.75      0.54      0.63      2170

avg / total       0.74      0.73      0.72      5244

Category:

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Use grid search to find better parameters
parameters = {
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=-1)

# Train the grid search model
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=

[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed: 13.5min remaining: 13.5min


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.2207123444428551, total=15.1min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.221173104434907, total=15.3min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.22443141181519097, total=15.3min
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.22672006865970534, total=16.2min
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.23390557939914164, total=16.3min
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.22800743813474467, total=16.3min


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 16.9min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
# Predict on test data using the best model
Y_pred_cv = cv.predict(X_test)

# Report the accuracy, precision, and recall of the tuned model
for i, column in enumerate(Y.columns):
    print(f'Category: {column}\n', classification_report(Y_test[column], Y_pred_cv[:, i]))

## 8. Try improving your model further

# Ideas for improvement:
# - Try different machine learning algorithms like AdaBoost or Gradient Boosting
# - Add other features such as POS tagging, named entity recognition, or using pre-trained word embeddings

## 9. Export your model as a pickle file

# Export the model to a pickle file
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

Category: related
              precision    recall  f1-score   support

          0       0.69      0.33      0.45      1205
          1       0.82      0.95      0.88      4005
          2       0.42      0.29      0.34        34

avg / total       0.79      0.81      0.78      5244

Category: request
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      4317
          1       0.85      0.40      0.55       927

avg / total       0.88      0.88      0.86      5244

Category: offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5226
          1       0.00      0.00      0.00        18

avg / total       0.99      1.00      0.99      5244

Category: aid_related
              precision    recall  f1-score   support

          0       0.75      0.87      0.80      3074
          1       0.76      0.59      0.66      2170

avg / total       0.75      0.75      0.74      5244

Category:

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [11]:
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.